In [ ]:
!pip install google-cloud-bigquery pandas
!pip install pandas_gbq

In [3]:
import os
import pandas as pd
import numpy as np
import pandas_gbq
import sqlite3

from google.cloud import bigquery
from google.oauth2 import service_account

In [4]:
service_path = "C:\\Users\\jandr\\OneDrive\\Desktop\\ADA\\wedge-project\\"
service_file = 'umt-msba-gg-key.json' 

gbq_proj_id = 'umt-msba'  

# And this should stay the same. 
private_key = service_path + service_file
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(private_key)
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
for item in client.list_datasets() : 
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


In [8]:
query='''
SELECT
EXTRACT(YEAR FROM datetime) AS year,
EXTRACT(HOUR FROM datetime) AS hour,
SUM(total) AS total_spend,
COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING),
CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) AS trans,
SUM(
    CASE WHEN trans_status IN ('V', 'R') THEN -1
        ELSE 1
    END) AS items
FROM `umt-msba.wedge_transactions.transArchive*`
WHERE department NOT IN (0, 15) AND (trans_status IS NULL OR trans_status IN (' ', 'V', 'R'))
GROUP BY year, hour
ORDER BY year, hour
'''

In [9]:
conn = sqlite3.connect('reporting.db')

In [10]:
holder= pandas_gbq.read_gbq(query, project_id=gbq_proj_id, credentials=credentials)

Downloading: 100%|██████████|


In [ ]:
#holder.head()

In [11]:
holder.to_sql("date_hour", conn, if_exists="replace", index=False)

141

In [12]:
# Query for Sales by owner by year by month
query_owner_year_month = '''
SELECT
  card_no,
  EXTRACT(YEAR FROM datetime) AS year,
  EXTRACT(MONTH FROM datetime) AS month,
  SUM(total) AS sales,
  COUNT(DISTINCT trans_id) AS transactions,
  SUM(CASE WHEN trans_status IN ('V', 'R') THEN -quantity ELSE quantity END) AS items
FROM `umt-msba.wedge_transactions.transArchive*`
WHERE department NOT IN (0, 15) AND (trans_status IS NULL OR trans_status IN (' ', 'V', 'R'))
GROUP BY card_no, year, month
'''


In [13]:
holder_owner_year_month = pandas_gbq.read_gbq(query_owner_year_month, project_id=gbq_proj_id, credentials=credentials)
holder_owner_year_month.to_sql("sales_owner_year_month", conn, if_exists="replace", index=False)

Downloading: 100%|██████████|


808896

In [26]:
#Query for Sales by product description by year by month
query_prod_year_month = '''
SELECT
  upc,
  description,
  department AS department_number,
  (SELECT dept_name FROM `umt-msba.wedge_transactions.department_lookup` as A WHERE department.D = department_number.A) AS department_name,
  EXTRACT(YEAR FROM datetime) AS year,
  EXTRACT(MONTH FROM datetime) AS month,
  SUM(total) AS sales,
  COUNT(DISTINCT trans_id) AS transactions,
  SUM(CASE WHEN trans_status IN ('V', 'R') THEN -quantity ELSE quantity END) AS items
FROM `umt-msba.wedge_transactions.transArchive*` as D
WHERE department NOT IN (0, 15) AND (trans_status IS NULL OR trans_status IN (' ', 'V', 'R'))
GROUP BY upc, description, department_number, department_name, year, month
'''

In [29]:
holder_prod_year_month = pandas_gbq.read_gbq(query_prod_year_month, project_id=gbq_proj_id, credentials=credentials)
holder_prod_year_month.to_sql("sales_prod_year_month", conn, if_exists="replace", index=False)

Downloading: 100%|██████████|


1184028

In [30]:
conn.close

<function Connection.close()>